# [백준/Euclid](https://www.acmicpc.net/problem/19171)


## 풀이과정


### 첫번째 시도


이 문제는 3차원의 세 점이 주어졌을 때, 이 세 점과 어떤 점 $X$ 간의 거리의 합을 $f(X)$라고 했을 때 $f(X)$ 의 최소값을 구하는 문제이다.
세 점을 각각 $A$, $B$, $C$라고 하면 $f(X)$는 다음과 같다.

$$
f(X) = |X - A| + |X - B| + |X - C| = \sqrt{(X - A)^2} + \sqrt{(X - B)^2} + \sqrt{(X - C)^2}
$$

$f(X)$는 $X$의 위치에 따라 달라지므로, $X$의 위치를 바꿔가며 $f(X)$를 계산하여 최소값을 구하는 방법으로 접근할 수 있다.
즉 $f(X)$의 변화율, $\nabla f(X)$를 구해야 한다.
한 번에 다 계산하기는 복잡하므로 $\frac{\partial |X - A|}{\partial x}$ 만 먼저 구해보자.

$$
\frac{\partial |X - A|}{\partial x} \\
 =\frac{\partial \sqrt{(x - A_x)^2 + (y - A_y)^2 + (z - A_z)^2}}{\partial x} \\
= \frac{d \sqrt{(x - A_x)^2}}{d x} = \frac{d (x - A_x)^2}{d x}\cdot\frac{d ((x - A_x)^2)^{1/2}}{d (x - A_x)^2} \\
= 2(x - A_x) \cdot\ 1/2\cdot((x - A_x)^2)^{-1/2} = \frac{x - A_x}{\sqrt{(x - A_x)^2}} \\
= \frac{x - A_x}{|x - A_x|}
$$

그럼 당연히

$$
\frac{\partial |X - A|}{\partial y} = \frac{y - A_y}{|y - A_y|} \\
\frac{\partial |X - A|}{\partial z} = \frac{z - A_z}{|z - A_z|}
$$

이므로

$$
\nabla |X - A| \\
= \left(\frac{\partial |X - A|}{\partial x}, \frac{\partial |X - A|}{\partial y}, \frac{\partial |X - A|}{\partial z}\right) \\
= \left(\frac{x - A_x}{|x - A_x|}, \frac{y - A_y}{|y - A_y|}, \frac{z - A_z}{|z - A_z|}\right) \\
= \frac{X - A}{|X - A|}
$$

이다.
여기까지 하면 대충 눈치 챘겠지만 최종적으로 $\nabla f(X) = \frac{X - A}{|X - A|} + \frac{X - B}{|X - B|} + \frac{X - C}{|X - C|}$ 이다.

서론이 길었는데 아무튼 $\nabla f(X)$ 방향으로 조금씩 이동하는, 경사 하강법을 통해 휴리스틱으로 $f(X)$의 최소값을 구할 수 있다.

기하 문제이므로 먼저 3차원 내 점을 정의하기 위한 클래스를 정의했다.
이 때 단순히 값만 저장하는게 아니라 메소드 오버로딩을 이용해 벡터 연산을 용이하게 만들었다.

```python
from math import hypot

class Point:
    def __init__(
        self, x: float | str | Iterable[float] = 0.0, y: float = 0.0, z: float = 0.0
    ):
        if isinstance(x, str):
            x, y, z = map(float, x.split())
        elif isinstance(x, Iterable):
            x, y, z = x
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)

    def __iter__(self) -> Generator[float, None, None]:
        yield from (self.x, self.y, self.z)

    def __sub__(self, other: "Point") -> "Point":
        return Point(map(lambda a, b: a - b, self, other))

    def __add__(self, other: "Point") -> "Point":
        return Point(map(lambda a, b: a + b, self, other))

    def __rmul__(self, other: float) -> "Point":
        return Point(map(lambda a: a * other, self))

    def __truediv__(self, other: float) -> "Point":
        return Point(map(lambda a: a / other, self))

    def __abs__(self) -> float:
        return hypot(*self)

    def grad(self) -> "Point":
        return self / abs(self) if abs(self) else self
```

여기서 `grad` 메소드는 $\nabla f(X)$를 구할 때 사용할 메소드이다.

```python
def sum_points(point0: Iterable[Point] | Point, *points: Point) -> Point:
    if isinstance(point0, Point):
        return sum(points, start=point0)
    return sum(point0, start=Point())

def grad_f(X: Point, A: Point, B: Point, C: Point) -> Point:
    return sum_points((X - P).grad() for P in [A, B, C])
```

`sum_points`는 `Point` 용 `sum` 이고 `grad_f`는 $\nabla f(X)$를 구하는 함수이다.

이제 경사 하강법을 구현해보자.
처음에는 다음과 같이 구현했었다.

```python
def minimize_dists(
    A: Point, B: Point, C: Point, init: Point, err: float, max_iter=1000
):
    X = init
    for i in range(max_iter):
        g = grad_f(X, A, B, C)
        if abs(g) < err:
            break
        X = X - err * g.grad()
    return X
```

처음에는 에러를 고정값으로 주고, 에러를 학습률로 쓰면서 최대 1000번 반복하도록 했다.
하지만 값이 커지니 에러가 클 때는 변화가 없다가 에러가 작아지면 급격히 변화가 돼서 최대 반복 횟수를 초과하는 경우가 생겼다.
그래서 손실을 계산한 뒤에 학습률을 조정하는 방법으로 바꿨다.

```python
def minimize_dists(
    A: Point, B: Point, C: Point, init: Point, err: float, max_iter=1000
):
    X = init
    prev_loss = float("inf")
    lr = err
    for _ in range(max_iter):
        g = grad_f(X, A, B, C)
        if abs(g) < err:
            break
        new_X = X - lr * g.grad()
        loss = sum(abs(new_X - P) for P in [A, B, C])
        if loss < prev_loss:
            # 손실이 줄어들면 학습률을 늘린다.
            X = new_X
            prev_loss = loss
            lr *= 1.05
        else:
            # 손실이 늘거나 변화가 없으면 학습률을 줄인다.
            lr *= 0.5
    return X
```

그리고 다음과 같은 `minimize_dists_recursive` 함수를 정의했다.

```python
def minimize_dists_recursive(
    A: Point,
    B: Point,
    C: Point,
):
    max_value = max(abs(A), abs(B), abs(C))
    iters = int(max_value).bit_length() // 3 # log_2(x) / 3 = log_8(x) ≈ log_10
    X = sum_points(A, B, C) / 3
    for i in range(iters, -5, -1):
        X = minimize_dists(A, B, C, X, 10**i)
    return sum(abs(X - P) for P in [A, B, C])
```

무게중심을 초깃값으로 두고 에러를 10의 거듭제곱으로 줄여가며 `minimize_dists`를 적용해 점을 구해서 최솟값을 구했다.
시간이 304ms 정도로 나와서 아쉬웠다.
백준 풀 때 클래스를 쓰면 시간이 확실히 많이 드는 것 같다.
그치만 3차원 기하학에서 벡터 연산 쓸 때는 `Point` 클래스를 정의해서 쓰는게 훨씬 편해서 이렇게 했다.


#### 풀이과정


In [ ]:
from math import hypot
from typing import Generator, Iterable


class Point:
    def __init__(
        self, x: float | str | Iterable[float] = 0.0, y: float = 0.0, z: float = 0.0
    ):
        if isinstance(x, str):
            x, y, z = map(float, x.split())
        elif isinstance(x, Iterable):
            x, y, z = x
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)

    def __iter__(self) -> Generator[float, None, None]:
        yield from (self.x, self.y, self.z)

    def __sub__(self, other: "Point") -> "Point":
        return Point(map(lambda a, b: a - b, self, other))

    def __add__(self, other: "Point") -> "Point":
        return Point(map(lambda a, b: a + b, self, other))

    def __rmul__(self, other: float) -> "Point":
        return Point(map(lambda a: a * other, self))

    def __truediv__(self, other: float) -> "Point":
        return Point(map(lambda a: a / other, self))

    def __abs__(self) -> float:
        return hypot(*self)

    def grad(self) -> "Point":
        return self / abs(self) if abs(self) else self


def sum_points(point0: Iterable[Point] | Point, *points: Point) -> Point:
    if isinstance(point0, Point):
        return sum(points, start=point0)
    return sum(point0, start=Point())


def grad_f(X: Point, A: Point, B: Point, C: Point) -> Point:
    return sum_points((X - P).grad() for P in [A, B, C])


def minimize_dists(
    A: Point, B: Point, C: Point, init: Point, err: float, max_iter=1000
):
    X = init
    prev_loss = float("inf")
    lr = err
    for _ in range(max_iter):
        g = grad_f(X, A, B, C)
        if abs(g) < err:
            break
        new_X = X - lr * g.grad()
        loss = sum(abs(new_X - P) for P in [A, B, C])
        if loss < prev_loss:
            X = new_X
            prev_loss = loss
            lr *= 1.05
        else:
            lr *= 0.5
    return X


def minimize_dists_recursive(
    A: Point,
    B: Point,
    C: Point,
):
    max_value = max(abs(A), abs(B), abs(C))
    iters = int(max_value).bit_length() // 3
    X = sum_points(A, B, C) / 3
    for i in range(iters, -5, -1):
        X = minimize_dists(A, B, C, X, 10**i)
    return sum(abs(X - P) for P in [A, B, C])


def solution():
    import sys

    points = list(map(Point, sys.stdin.read().strip().split("\n")))
    print("%.6f" % abs(minimize_dists_recursive(*points)))


solution()

## 해답


In [1]:
from math import hypot
from typing import Generator, Iterable

In [2]:
class Point:
    def __init__(
        self, x: float | str | Iterable[float] = 0.0, y: float = 0.0, z: float = 0.0
    ):
        if isinstance(x, str):
            x, y, z = map(float, x.split())
        elif isinstance(x, Iterable):
            x, y, z = x
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)

    def __iter__(self) -> Generator[float, None, None]:
        yield from (self.x, self.y, self.z)

    def __sub__(self, other: "Point") -> "Point":
        return Point(map(lambda a, b: a - b, self, other))

    def __add__(self, other: "Point") -> "Point":
        return Point(map(lambda a, b: a + b, self, other))

    def __rmul__(self, other: float) -> "Point":
        # scalar
        return Point(map(lambda a: a * other, self))

    def __truediv__(self, other: float) -> "Point":
        return Point(map(lambda a: a / other, self))

    def __abs__(self) -> float:
        return hypot(*self)

    def __repr__(self) -> str:
        return f"Point({self.x:.2f}, {self.y:.2f}, {self.z:.2f})"

    def grad(self) -> "Point":
        return self / abs(self) if abs(self) else self

In [ ]:
def sum_points(point0: Iterable[Point] | Point, *points: Point) -> Point:
    if isinstance(point0, Point):
        return sum(points, start=point0)
    return sum(point0, start=Point())


def grad_f(X: Point, A: Point, B: Point, C: Point) -> Point:
    return sum_points((X - P).grad() for P in [A, B, C])


def minimize_dists(
    A: Point, B: Point, C: Point, init: Point, err: float, max_iter=1000
):
    X = init
    prev_loss = float("inf")
    lr = err
    for i in range(max_iter):
        g = grad_f(X, A, B, C)
        if abs(g) < err:
            break
        new_X = X - lr * g.grad()
        loss = sum(abs(new_X - P) for P in [A, B, C])
        if loss < prev_loss:
            X = new_X
            prev_loss = loss
            lr *= 1.05  # 증가
        else:
            lr *= 0.5  # 감소
    print(f"{err = } iter {i}: X = {X}, g = {g}")
    return X


def minimize_dists_recursive(
    A: Point,
    B: Point,
    C: Point,
):
    max_value = max(abs(A), abs(B), abs(C))
    iters = int(max_value).bit_length() // 3
    X = sum_points(A, B, C) / 3
    for i in range(iters, -5, -1):
        X = minimize_dists(A, B, C, X, 10**i)
    return sum(abs(X - P) for P in [A, B, C])

In [4]:
def solution():
    import sys

    points = list(map(Point, sys.stdin.read().strip().split("\n")))
    print("%.6f" % abs(minimize_dists_recursive(*points)))

## 예제


In [5]:
# 백준 문제 풀이용 예제 실행 코드
from bwj import test

test_solution = test(solution)

# test_solution("""""")
# test_solution(read("fn").read())

In [6]:
test_solution(
    """1000000000 1000000000 1000000000
0 -500000000 -1000000000
0 -1000000000 -1000000000
"""
)  # 3192582403.567255

err = 10000000000 iter 0: X = Point(333333333.33, -166666666.67, -333333333.33), g = Point(0.35, 0.54, 0.71)
err = 1000000000 iter 0: X = Point(333333333.33, -166666666.67, -333333333.33), g = Point(0.35, 0.54, 0.71)
err = 100000000 iter 0: X = Point(333333333.33, -166666666.67, -333333333.33), g = Point(0.35, 0.54, 0.71)
err = 10000000 iter 0: X = Point(333333333.33, -166666666.67, -333333333.33), g = Point(0.35, 0.54, 0.71)
err = 1000000 iter 0: X = Point(333333333.33, -166666666.67, -333333333.33), g = Point(0.35, 0.54, 0.71)
err = 100000 iter 0: X = Point(333333333.33, -166666666.67, -333333333.33), g = Point(0.35, 0.54, 0.71)
err = 10000 iter 0: X = Point(333333333.33, -166666666.67, -333333333.33), g = Point(0.35, 0.54, 0.71)
err = 1000 iter 0: X = Point(333333333.33, -166666666.67, -333333333.33), g = Point(0.35, 0.54, 0.71)
err = 100 iter 0: X = Point(333333333.33, -166666666.67, -333333333.33), g = Point(0.35, 0.54, 0.71)
err = 10 iter 0: X = Point(333333333.33, -166666666.67,

In [7]:
test_solution(
    """0 0 0
2000000 0 0
1000000 2000000 0
"""
)  # 3732050.812789

err = 10000000 iter 0: X = Point(1000000.00, 666666.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 1000000 iter 0: X = Point(1000000.00, 666666.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 100000 iter 0: X = Point(1000000.00, 666666.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 10000 iter 0: X = Point(1000000.00, 666666.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 1000 iter 0: X = Point(1000000.00, 666666.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 100 iter 0: X = Point(1000000.00, 666666.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 10 iter 0: X = Point(1000000.00, 666666.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 1 iter 0: X = Point(1000000.00, 666666.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 0.1 iter 171: X = Point(1000000.00, 658266.33, 0.00), g = Point(0.00, 0.10, 0.00)
err = 0.01 iter 263: X = Point(1000000.00, 583481.33, 0.00), g = Point(0.00, 0.01, 0.00)
err = 0.001 iter 257: X = Point(1000000.00, 577900.76, 0.00), g = Point(0.00, 0.00, 0.00)
err = 0.0001 iter 254: X = Point(

In [8]:
test_solution(
    """0 0 0
2000 0 0
1000 2000 0
"""
)  # 3732.050813

err = 10000 iter 0: X = Point(1000.00, 666.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 1000 iter 0: X = Point(1000.00, 666.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 100 iter 0: X = Point(1000.00, 666.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 10 iter 0: X = Point(1000.00, 666.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 1 iter 0: X = Point(1000.00, 666.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 0.1 iter 34: X = Point(1000.00, 658.16, 0.00), g = Point(0.00, 0.10, 0.00)
err = 0.01 iter 122: X = Point(1000.00, 581.42, 0.00), g = Point(0.00, 0.01, 0.00)
err = 0.001 iter 105: X = Point(1000.00, 578.09, 0.00), g = Point(0.00, 0.00, 0.00)
err = 0.0001 iter 119: X = Point(1000.00, 577.43, 0.00), g = Point(0.00, 0.00, 0.00)
3732.050811


In [9]:
test_solution(
    """0 0 0
200 0 0
100 200 0
"""
)  # 373.205081

err = 100 iter 0: X = Point(100.00, 66.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 10 iter 0: X = Point(100.00, 66.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 1 iter 0: X = Point(100.00, 66.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 0.1 iter 7: X = Point(100.00, 65.85, 0.00), g = Point(0.00, 0.10, 0.00)
err = 0.01 iter 75: X = Point(100.00, 58.29, 0.00), g = Point(0.00, 0.01, 0.00)
err = 0.001 iter 66: X = Point(100.00, 57.81, 0.00), g = Point(0.00, 0.00, 0.00)
err = 0.0001 iter 72: X = Point(100.00, 57.74, 0.00), g = Point(0.00, 0.00, 0.00)
373.205081


In [10]:
test_solution(
    """0 0 0
20 0 0
10 20 0
"""
)  # 37.320508

err = 10 iter 0: X = Point(10.00, 6.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 1 iter 0: X = Point(10.00, 6.67, 0.00), g = Point(0.00, 0.11, 0.00)
err = 0.1 iter 1: X = Point(10.00, 6.57, 0.00), g = Point(0.00, 0.10, 0.00)
err = 0.01 iter 32: X = Point(10.00, 5.81, 0.00), g = Point(0.00, 0.01, 0.00)
err = 0.001 iter 20: X = Point(10.00, 5.78, 0.00), g = Point(0.00, 0.00, 0.00)
err = 0.0001 iter 30: X = Point(10.00, 5.77, 0.00), g = Point(0.00, 0.00, 0.00)
37.320508
